# Week1_Library 과제

### Q1. Library 와 Framework 의 차이 간단하게 서술하시오. (100자 내외)

라이브러리와 프레임워크는 이미 검증된 코드와 시스템 및 설계 위에서 코딩을 하여 시간을 단축하고 통일성이 생긴다는 공통점이 존재한다. 하지만 프레임워크는 하나의 작업공간이며 주어지는 도구들만을 사용하여 코드를 구축해나가야한다. 반대로 라이브러리같은 경우는 본인이 코드를 칠 때 하나의 도구로서 활용이 가능한 것을 의미한다.

### Q2. 딥러닝과 머신러닝의 관계 및 특징, 차이 간단하게 서술하시오. (200자 내외)

머신러닝은 데이터를 기반으로 학습하여 추론하는 기술이며, 딥러닝은 머신러닝의 기술 중 하나이다. 하지만 딥러닝과 머신러닝에는 확실한 차이가 있는데 바로 사람의 개입이 들어가는지에 대한 유무이다. 머신러닝의 경우에는 컴퓨터가 학습하기전 여러가지 특징들을 미리 전처리 과정을 통해 분석해야 한다. 반대로 딥러닝의 경우에는 따로 패턴을 분석하고 특징을 추출하는 사람의 개입이 없어지며, 만들어진 신경망에 데이터를 입력하여 예측하도록 한다.

### Q3. 아래의 코드에 주석 달기.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms

# 사용 가능한 gpu가 있을 때만 cuda 연산 할당
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 시드 고정
torch.manual_seed(45)
if device == 'cuda':
    torch.cuda.manual_seed_all(45)
print(device + " is available")

In [ ]:
learning_rate = 0.001
batch_size = 100 
num_classes = 10 # 총 분류할 클래스의 개수 : 10개
epochs = 5

In [ ]:
# 학습 데이터 불러오기(MNIST)
train_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = True,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor() 
    ])
)

# 테스트 데이터 불러오기(MNIST)
test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor()
    ])
)

In [ ]:
# 데이터 로더 생성
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

In [ ]:
# 모델 클래스 생성
class ConvNet(nn.Module):
    def __init__(self): 
        super(ConvNet, self).__init__() 
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) # input_channel : 1, output_channel : 10, kernel_size : 5
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) 
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False) # 0.25% 만큼 dropout
        self.mp = nn.MaxPool2d(2)  # 0.5 만큼 size 축소
        self.fc1 = nn.Linear(320,100)  # input_channel : 320, output_channel : 100
        self.fc2 = nn.Linear(100,10) # 출력되는 channel 개수 = class 개수


    def forward(self, x):
        x = F.relu(self.mp(self.conv1(x))) # convolution -> maxpooling -> ReLu
        x = F.relu(self.mp(self.conv2(x))) # convolution -> maxpooling -> ReLu
        x = self.drop2D(x) # dropout
        x = x.view(x.size(0), -1)  # flatten
        x = self.fc1(x)  # fcLayer
        x = self.fc2(x)  # fcLayer
        return F.log_softmax(x)  # softmax

In [ ]:
model = ConvNet().to(device) # model class -> device 환경
criterion = nn.CrossEntropyLoss().to(device)  # 손실함수 : Cross Entropy -> device 환경
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)  # Optimizer : Adam

In [ ]:
# train
for epoch in range(epochs): 
    avg_cost = 0
    for data, target in train_loader:
        data = data.to(device) # data -> device
        target = target.to(device)  # label -> device
        optimizer.zero_grad() # gradient 0으로 초기화
        hypothesis = model(data) # data -> model
        cost = criterion(hypothesis, target) # 손실함수 계산(output, target)
        cost.backward() # 가중치와 편향에 대한 기울기 계산
        optimizer.step()  # 기울기를 이용하여 업데이트
        avg_cost += cost / len(train_loader)  # loss 값 평균
    print('[Epoch: {:>4}]  cost = {:>.9}'.format(epoch + 1, avg_cost))

In [ ]:
# Test
model.eval()  # test 작업 시작
with torch.no_grad(): 
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device)  # data -> device
        target = target.to(device)  # target -> device
        out = model(data)  # data -> model
        preds = torch.max(out.data, 1)[1]  # predict
        total += len(target) # target 개수 (누적)
        correct += (preds==target).sum().item() # 예측값이 정답인 경우 (누적)
        
    print('Test Accuracy: ', 100.*correct/total, '%') # 정확도 계산

## 첫 정규세션 들으시느라 고생 많으셨습니다.